## Connection to Thymio

In [1]:
# Connect to Thymio with ClientAsync
from tdmclient import ClientAsync, aw
client = ClientAsync()
node = await client.wait_for_node()

# Lock Thymio
aw(node.lock())

Node 52ceb7f5-6164-4bf6-b28a-33899f775678

## Importation of libraries

In [ ]:
from kalman_filtering.kalman_filter import *
from thymio.control import *
from computer_vision.vision import *

## Main script

In [ ]:
u_old = 0
mu_predict_old = 0
cov_predict_old = 0
view_blocked = False
active = True
end = False
start = True
path = None
segment_index = 0
obstacle = 0

map = Map()

while active == True:
    active = await stop_thymio(node)

    map.update()
    map.show()
    
    # State variables measurement
    await node.wait_for_variables()
    vel_meas = velocity_with_odometry(node, map.robot[2])

    # Filtering
    y = np.array([map.robot[0]], [map.robot[1]], [vel_meas[0]], [vel_meas[1]])
    x_est, cov_est = kalman_filter(y, u_old, mu_predict_old, cov_predict_old, view_blocked)
    pose_est = x_est[0:2]
    pose_est.append(map.robot[2])
    error_est = cov_est[0:2]
    error_est.append(0)

    # Global Navigation
    #create path
    if start == True:
        ex_path = compute_visibility_matrix(map.obstacles) 
        path_incomplete = a_star_search(pose_est,ex_path)
        path = path_functions(path_incomplete)
        start = False


    # Local Navigation
    #u, obstacle, segment_index = collision_avoidance(path, pose_est, error_est, old_obstacle, segment_index) # Zacharie

    # Global Navigation
    if not obstacle:
        u, segment_index, end = path_direction(pose_est, path, segment_index)


    motors(u[0], u[1])

    u_old = u
    cov_predict_old = cov_est
    mu_predict_old = x_est
    obstacle_old = obstacle

    if cv.waitKey(1) != -1:
        map.vision_stop()
        break

map.info()
cv.destroyAllWindows()


NameError: name 'Map' is not defined

## Motors control test

In [10]:
active = True
vel_meas = []

while active:
    await node.wait_for_variables()
    vel_meas.append(velocity_with_odometry(node, 0)[0])
    await node.set_variables(motors(-100, 100))
    active = await stop_thymio(node)

await node.set_variables(motors(0, 0)) 

vel_meas_mean = np.mean(vel_meas[-100:])
vel_meas_var = np.var(vel_meas[-100:])
print("Velocity measured:", vel_meas)
print("Velocity mean:", vel_meas_mean)
print("Velocity variance:", vel_meas_var)

Center button pressed. Stopping the algorithm.
Velocity measured: [97.5, 2.5, -3.0, -3.0, -3.0, -4.0, -4.0, 6.0, 6.0, 0.5, 0.5, 0.0, 0.0, 6.5, 6.5, -9.0, -9.0, 5.5, 5.5, 5.5, 0.5, 0.5, 2.0, 2.0, 3.5, 3.5, -4.0, -4.0, -4.0, -2.0, -2.0, -3.5, -3.5, -0.5, -0.5, 1.5, 1.5, 2.0, 2.0, -5.0, -5.0, -5.0, 2.0, 2.0, -5.0, -5.0, -5.0, -6.0, -6.0, 3.0, 3.0, -1.0, -1.0, 5.0, 5.0, -3.5, -3.5, -8.0, -8.0, -3.0, -3.0, 2.0, 2.0, 2.0, 4.0, 4.0, 3.0, 3.0, 1.0, 1.0, -2.0, -2.0, -1.0, -1.0, -1.5, -1.5, 3.5, 3.5, 3.5, -3.0, -3.0, -3.0, -3.0, 0.5, 0.5, 5.5, 5.5, 4.0, 4.0, 6.0, 3.5, 3.5, 3.5, -3.0, -3.0, 3.5, 3.5, -0.5, -0.5, 4.0, 4.0, 4.0, -3.0, -3.0, -1.0, -1.0, -5.5, -5.5, 5.5, 5.5, 2.0, 2.0, 4.5, 4.5, 8.5, 8.5, 3.0, 3.0, 9.5, 9.5, -6.5, -6.5, -6.5, -6.5, -6.5, -9.0, -9.0, -6.0, -6.0, -4.5, -4.5, -5.5, -5.5, 1.5, 1.5, 1.0, 1.0, -5.5, -5.5, -3.5, -3.5, -3.5, 1.0, 1.0, 3.5, 3.5, 1.0, 1.0, -6.5, -6.5, -6.5, -9.0, -9.0, 7.0, 1.5, 1.5, 10.5, 10.5, 2.5, 2.5, 7.0, 7.0, 4.5, 4.5, -2.0, -2.0, -7.5, -7.5, -7.5, -1.5,

Exception in thread Thread-3:
Traceback (most recent call last):
  File "c:\Users\samyg\AppData\Local\Programs\Python\Python310\lib\threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "c:\Users\samyg\AppData\Local\Programs\Python\Python310\lib\site-packages\tdmclient\tcp.py", line 73, in run
    packet = self.read_packet()
  File "c:\Users\samyg\AppData\Local\Programs\Python\Python310\lib\site-packages\tdmclient\tcp.py", line 66, in read_packet
    raise error
  File "c:\Users\samyg\AppData\Local\Programs\Python\Python310\lib\site-packages\tdmclient\tcp.py", line 59, in read_packet
    packet_len = self.read_uint32()
  File "c:\Users\samyg\AppData\Local\Programs\Python\Python310\lib\site-packages\tdmclient\tcp.py", line 46, in read_uint32
    b = self.io.read(4)
  File "c:\Users\samyg\AppData\Local\Programs\Python\Python310\lib\site-packages\tdmclient\tcp.py", line 99, in read
    return self.socket.recv(n)
ConnectionResetError: [WinError 10054] An existing connection w

In [6]:
node.var

{'_fwversion': [14, 0],
 '_id': [-1010],
 '_imot': [-200, -186],
 '_integrator': [-1052, -1048],
 '_productId': [8],
 '_vbat': [747, 753],
 'acc': [0, 0, 20],
 'acc._tap': [82],
 'button.backward': [0],
 'button.center': [1],
 'button.forward': [0],
 'button.left': [0],
 'button.right': [0],
 'buttons._mean': [11080, 9442, 14275, 14186, 15623],
 'buttons._noise': [66, 42, 51, 46, 62],
 'buttons._raw': [11072, 9439, 13535, 14190, 15621],
 'event.args': [0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 'event.source': [-1010],
 'leds.bottom.left': [0, 0, 0],
 'leds.bottom.right': [0, 0, 0],
 'leds.circle': [0, 0, 0, 0, 0, 0, 0, 0],
 'leds.top': [0, 0, 0],
 'mic._mean': [257],
 'mic.intensity': [111],
 'mic.threshold': [0],
 'motor.left.pwm': [-304],
 'motor.left.speed': [108],
 'motor.left.target': [100],
 'motor.right.pwm': [-328],
 'motor.right.speed': [95],
 'motor.right.targ